# ArcGIS-Enhanced Appalachian Trail Analysis

This notebook demonstrates advanced analyses using real ArcGIS data from the official AT NRCA Hub.

## Analyses Included

1. **Shelter Spacing Analysis** - Optimal daily mileage planning
2. **Water Availability** - Identify dry sections and water gaps
3. **Resource Density Mapping** - Clustering of shelters, water, parking
4. **Difficulty vs Support Analysis** - Do harder sections have more resources?
5. **State Infrastructure Comparison** - Which states have better support?
6. **Optimal Resupply Planning** - Town access and distances

---

**Data Source**: [AT NRCA Datasets](https://appalachian-trail-natural-resource-condition-assessment-clus.hub.arcgis.com/search?collection=dataset)


## Setup & Imports


In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, './src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from data_loader import load_or_generate_data
from analysis import TrailAnalyzer
from arcgis_integration import ArcGISDataFetcher

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Modules loaded successfully")


✓ Modules loaded successfully


## Load Data

Try ArcGIS data first, fall back to synthetic if unavailable.


In [2]:
# Try loading ArcGIS data
print("Checking for ArcGIS data...")
fetcher = ArcGISDataFetcher()
arcgis_data = fetcher.create_enhanced_dataset(use_cache=True)

if arcgis_data is not None and len(arcgis_data) > 0:
    df = arcgis_data
    data_source = "ArcGIS (Real Data)"
    print(f"✓ Using real ArcGIS data: {len(df)} points")
else:
    print("⚠ Using synthetic data (ArcGIS data not available)")
    df, _ = load_or_generate_data('./data/trail_data.csv')
    data_source = "Synthetic"
    print(f"✓ Loaded: {len(df)} points")

print(f"\nData: {data_source}")
print(f"Distance: {df['distance_miles'].max():.1f} miles")
print(f"Elevation: {df['elevation_ft'].min():.0f} - {df['elevation_ft'].max():.0f} ft")

df.head()


Checking for ArcGIS data...

CREATING ENHANCED DATASET WITH ARCGIS DATA

Step 1: Fetching trail centerline...
Fetching AT centerline from ArcGIS...
⚠ API returned status 400

⚠ Could not fetch real ArcGIS data.
  Falling back to synthetic data generation.
  To use real data, check ArcGIS Hub for current service endpoints:
  - https://hub.arcgis.com
  - Search for 'Appalachian Trail Conservancy'
⚠ Using synthetic data (ArcGIS data not available)
✓ Loaded: 19560 points

Data: Synthetic
Distance: 1955.9 miles
Elevation: 0 - 4854 ft


,point_id,latitude,longitude,elevation_ft,distance_miles,state
0,0,34.628027,-84.187432,3795.884701,0.0,Georgia
1,1,34.629003,-84.202235,3791.055301,0.1,Georgia
2,2,34.630778,-84.175789,3797.315228,0.2,Georgia
3,3,34.617059,-84.207191,3821.714640,0.3,Georgia
4,4,34.630602,-84.206715,3834.013993,0.4,Georgia
